In [1]:
from vector_database.retrieve import retrieve_elements
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import torch
from rag_configuration import CFG
torch.set_default_device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
torch.cuda.is_available()

True

In [3]:
embedding_model = AutoModel.from_pretrained(CFG.embedding_model_name, trust_remote_code=True,
                                            torch_dtype=torch.bfloat16, use_flash_attn=False).to('cuda')
embedding_tokenizer = AutoTokenizer.from_pretrained(CFG.embedding_model_name)
reranker_tokenizer = AutoTokenizer.from_pretrained(CFG.reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(
    CFG.reranker_model_name, trust_remote_code=True,
    torch_dtype='auto')
embedding_tokenizer.model_max_length = 8192
reranker_tokenizer.model_max_length = 8192

In [4]:
queries = ['Welches Schmerzmittel sollte ich einem schwer verletztem Kind geben?', 'Was bedeutet es, wenn die Arme eines schwer verletzen Patienten anschwellen?', 'Worauf sollte man bei einem versuchten Selbstmord durch Abgase bei einem Patienten achten?']

In [5]:
test_query = queries[1]

In [6]:
elems = retrieve_elements(test_query, embedding_model)

OperationalError: connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [43]:
document_filename, titles, sections,headings, texts, table_flags, page_numbers, cosine_similarities = zip(*elems)

In [44]:
scores = reranker_model.compute_score([[test_query, text] for text in texts])

In [45]:
top_k_scores = scores.copy()
top_k_scores.sort()
top_k_scores = top_k_scores[::-1][:CFG.number_of_retrieved_documents]

In [46]:
top_k_texts = [text for text, score in zip(texts, scores) if score in top_k_scores]
for text in top_k_texts:
    print(50*'#')
    print(text)

##################################################
Toxikologie : Bei Verdacht auf beabsichtige oder suizidale Kohlenmonoxidvergiftung empfehlen Weaver et al. ein zusätzliches toxikologisches Screening, inklusive Blutethanolspiegel [35]. In einer Studie wurden bei 183/426 Patienten mit beabsichtiger Kohlenmonoxidvergiftung eine zusätzliche Intoxikation vor allem mit Alkohol nachgewiesen [61].
##################################################
Kohlenmonoxid kann in zahlreichen Situationen und Umgebungen auftreten. Typische Situationen sind Brandrauch, Bewusstseinsstörungen ohne erkennbare Ursache in geschlossenen Räumen mit Feuerstellen (z.B. Heizungen, Öfen, Kamin, Grill); Suizid(versuch) - oft mit entsprechenden Hinweisen (schriftliche Warnung, abgeklebte Räume); in Silos mit großen Mengen an Holzpellets; Motoren-Abgase (ohne Katalysator) und der Gebrauch von Wasserpfeifen.
##################################################
In der Notaufnahme soll insbesondere auch bei nicht durch den 